# Vaccine Dashboard Update
Center for Human Dynamics in the Mobile Age (HDMA) at San Diego State University <br>
Jessica Embury

### IMPORTS

In [69]:
#import modules
#import arcgis
from arcgis.gis import GIS
from arcgis import geometry
from arcgis.features import GeoAccessor, GeoSeriesAccessor
from arcgis.features import FeatureLayerCollection
from arcgis.features import FeatureLayer
#from arcgis.mapping import WebMap

from copy import deepcopy
import json
import numpy as np
import os
import pandas as pd
import requests
import sys
import tabula
import time
import webbrowser

#ignore pandas slice warning
pd.options.mode.chained_assignment = None  # default='warn'

### DATE, FILE PATHS, & AGOL IDS

In [70]:
###########################
###SET DATA THROUGH DATE###
###########################
most_recent_date = '03/13/2021'  # Change date daily using 'MM/DD/YYYY' format
max_cases = 13271  # max covid cases in any sd zip code, use until script merged with main notebook

##########################
###SET PATHS IN AND OUT###
##########################
#PATHS IN
pdf_path = 'C:/Users/jesse/Dropbox/Mapping-Vulearable-Pop-Tasks/SD-County-Data/{}/COVID-19 Vaccine Report by Zipcode.pdf'.format(most_recent_date.replace('/', '-'))

#additional needed zip info - community name and population
zips_info_path = './data/zip_coords_pop.csv'

#paths out
path_out = './data/sd_zip_vaccines.csv' #feature overwrite
path_out2 = './data/sd_zip_vaccines_alldates.csv' # date columns
path_out3 = '../covid_data/vaccines/sd_zip_vaccines_{}.csv'.format(most_recent_date.replace('/', '')) # overwrite data archive
path_out4 = './data/sd_zip_vaccines_7d.csv' #feature overwrite for 7-day layer
path_db = 'C:/Users/jesse/Dropbox/Mapping-Vulearable-Pop-Tasks/SD-County-Data/COVID-Data-Share-at-HDMA-Center/Vaccination/SD_Zip_COVID_Vaccine_{}.csv'.format(most_recent_date.replace('/', ''))

vacc_count_map_id = '3a5a90a0b43b42bb823ab4136b8d58e9'
conf_cases_map_id = '9b08e97e950f4cd79743e3a8217d4508'
vacc_dash = 'https://arcg.is/15ePOz'

vacc_7d_layer_id = 'dc90f581b4914e52b382b086423fb6b7'
vacc_7d_map_id = '19e5ea7a388c4eb18c70392b7d8f6a03'
vacc_7d_dash = 'https://arcg.is/b4iHr'

vacc_hpi_id = '694f95f80e6849608470e5ecfc1448d2'
vacc_hpi_dash = 'https://experience.arcgis.com/experience/16370187fbb64720b468483638c02e78/'

vacc_hpi_7d_id = '7ed91cc1f7454df6b750c784b7ce70d3'
vacc_hpi_7d_dash = ''

### CONVERT VACCINE PDF TO CSV FILES

In [ ]:
#read pdf
tables = tabula.read_pdf(pdf_path, pages = "all", multiple_tables = True)

In [ ]:
tables[0].head()

In [ ]:
#format 1st column zips
t1 = tables[0][[1, 2, 3]]
#rename columns
t1 = t1.rename(columns = {1: 'Zip Code', 2: 'Vacc Count', 3: 'Rate Per 1000'})
# drop bad rows
t1 = t1.drop(t1.index[0])
t1.head()

In [ ]:
#format 2nd column zips
t2 = tables[0][[4, 6, 8]]
#rename columns
t2 = t2.rename(columns = {4: 'Zip Code', 6: 'Vacc Count', 8: 'Rate Per 1000'})
# drop bad rows
t2 = t2.drop(t2.index[[0, 57]])
#t2.head()
t2.tail()

In [ ]:
# merge t1 and t2
t1 = t1.append(t2, ignore_index=True)
print(len(t1))

t1

In [ ]:
#format numbers in counts column
t1 = t1.fillna(0)

for i, row in t1.iterrows():
    try:
        t1['Vacc Count'][i] = str(t1['Vacc Count'][i].replace(',', '')).replace('*','')
        t1['Vacc Count'][i] = int(t1['Vacc Count'][i])
    except:
        print(t1['Zip Code'][i], t1['Vacc Count'][i])
    
    try:
        t1['Rate Per 1000'][i] = str(t1['Rate Per 1000'][i].replace('***', '0').replace('**', '0'))
        t1['Rate Per 1000'][i] = float(t1['Rate Per 1000'][i])
    except:
        print(t1['Zip Code'][i], t1['Rate Per 1000'][i])

t1['Vacc Count'] = t1['Vacc Count'].astype(int)
print(type(t1['Vacc Count'][0]))

t1['Rate Per 1000'] = t1['Rate Per 1000'].astype(float)
print(type(t1['Rate Per 1000'][0]))

t1.head()

In [ ]:
# supplemental zip code data
zips = pd.read_csv(zips_info_path)
zips.head()

In [ ]:
# merge vaccine info with zip code info
t1['Zip Code'] = t1['Zip Code'].astype(str)
zips['Zip'] = zips['Zip'].astype(str)

df = pd.merge(zips, t1, how = 'right', left_on = 'Zip', right_on = 'Zip Code')
del df['Zip Code']

# add data date
df['Date'] = most_recent_date
df

In [ ]:
# prep data for dated column csv
df2 = pd.read_csv(path_out2)
print(len(df2))
df2['Zip'] = df2['Zip'].astype(str)
df2 = df2.merge(df[['Zip', 'Vacc Count']], how = "right", on = "Zip")
print(len(df2))
df2 = df2.rename(columns = {'Vacc Count': most_recent_date})
df2.head()

In [ ]:
# calculate 7-day changes
df3 = df2[['Zip', df2.columns[-8], df2.columns[-1]]]
df3 = df3.fillna(0)
df3['7-Day Vacc Count'] = df3[df3.columns[2]].astype(int) - df3[df3.columns[1]].astype(int)

# add 7-day date range column
df3['7-Day Date Range'] = '{} - {}'.format(df2.columns[-7].replace('2021', '21'), df2.columns[-1].replace('2021', '21'))

del df3[df3.columns[1]]
del df3[df3.columns[1]]

df3.head()

In [ ]:
# merge 7-day vaccines to main df
df = df.merge(df3, how = 'left', on = 'Zip')
df.head()

In [ ]:
df['7-Day Rate Per 1000'] = 0.0

#7d rate per 1000
for i, row in df.iterrows():
    if df['2018_population'][i] >= 10000:
        df['7-Day Rate Per 1000'][i] = round(df['7-Day Vacc Count'][i]/df['2018_population'][i]*1000, 1)


In [ ]:
print(len(df))
df.tail()

In [ ]:
# save feature overwrite info
df.to_csv(path_out, index=False)
df.to_csv(path_out3, index=False)
df.to_csv(path_out4, index=False)

In [ ]:
# save dated columns to csv
df2.to_csv(path_out2, index=False)
df2.to_csv(path_db, index=False)

In [ ]:
# get max count and rate for map updates
max_count = df['Vacc Count'].max()
max_rate = df['Rate Per 1000'].max()
max_7d = df['7-Day Vacc Count'].max()
max_7d_rate = df['7-Day Rate Per 1000'].max()
print(max_count, max_rate, max_7d, max_7d_rate)

### CONNECT TO AGOL ACCOUNT

In [71]:
gis = GIS("pro")

### FUNCTIONS

In [95]:
def get_map (map_id):
    '''
    GET MAP DATA FOR SYMBOLOGY CHANGES
    '''
    m = gis.content.get(map_id)
    data = m.get_data()
    #Include the below line for prettified JSON
    #print(json.dumps(data, indent=4, sort_keys=True))
    print(m)
    
    return data
    
def update_map (map_id, data):
    '''
    UPDATE MAP TO SAVE CHANGES
    '''
    m = gis.content.get(map_id)
    # Set the item_properties to include the desired update
    properties = {"text": json.dumps(data)}
    # 'Commit' the updates to the Item
    update = m.update(item_properties=properties)
    
    return update

### FEATURE LAYER OVERWRITE

In [ ]:
#get feature layer containing updated data for maps associated with the 7-day COVID-19 vaccines
layer = gis.content.get(vacc_7d_layer_id)
layer

layer_collection = FeatureLayerCollection.fromitem(layer)

#call the overwrite() method which can be accessed using the manager property
layer_collection.manager.overwrite(path_out4)

### UPDATE MAP SYMBOLOGY

In [ ]:
# TOTAL VACC MAP
#get map data
vacc1_data = get_map(vacc_count_map_id)

#adjust symbology for graduated points to reflect new max/min
#MAX VACC COUNT
vacc1_data['operationalLayers'][2]['layerDefinition']['drawingInfo']['renderer']['authoringInfo']['visualVariables'][0]['maxSliderValue'] = max_count.item()
vacc1_data['operationalLayers'][2]['layerDefinition']['drawingInfo']['renderer']['visualVariables'][0]['maxDataValue'] = max_count.item()

vacc1_data['operationalLayers'][3]['layerDefinition']['drawingInfo']['renderer']['authoringInfo']['visualVariables'][0]['maxSliderValue'] = max_count.item()
vacc1_data['operationalLayers'][3]['layerDefinition']['drawingInfo']['renderer']['visualVariables'][0]['maxDataValue'] = max_count.item()

#update map to save changes
vacc1_update = update_map(vacc_count_map_id, vacc1_data)
vacc1_update

In [ ]:
# TOTAL COVID MAP
#get map data
vacc2_data = get_map(conf_cases_map_id)

#adjust symbology for graduated points to reflect new max/min
#MAX VACC COUNT
vacc2_data['operationalLayers'][2]['layerDefinition']['drawingInfo']['renderer']['authoringInfo']['visualVariables'][0]['maxSliderValue'] = max_cases
vacc2_data['operationalLayers'][2]['layerDefinition']['drawingInfo']['renderer']['visualVariables'][0]['maxDataValue'] = max_cases

vacc2_data['operationalLayers'][3]['layerDefinition']['drawingInfo']['renderer']['authoringInfo']['visualVariables'][0]['maxSliderValue'] = max_cases
vacc2_data['operationalLayers'][3]['layerDefinition']['drawingInfo']['renderer']['visualVariables'][0]['maxDataValue'] = max_cases

#update map to save changes
vacc2_update = update_map(conf_cases_map_id, vacc2_data)
vacc2_update

In [ ]:
# 7 DAY VACC MAP
#get map data
vacc3_data = get_map(vacc_7d_map_id)

#adjust symbology for graduated points to reflect new max/min
#MAX VACC COUNT
vacc3_data['operationalLayers'][2]['layerDefinition']['drawingInfo']['renderer']['authoringInfo']['visualVariables'][0]['maxSliderValue'] = max_7d.item()
vacc3_data['operationalLayers'][2]['layerDefinition']['drawingInfo']['renderer']['visualVariables'][0]['maxDataValue'] = max_7d.item()

vacc3_data['operationalLayers'][3]['layerDefinition']['drawingInfo']['renderer']['authoringInfo']['visualVariables'][0]['maxSliderValue'] = max_7d.item()
vacc3_data['operationalLayers'][3]['layerDefinition']['drawingInfo']['renderer']['visualVariables'][0]['maxDataValue'] = max_7d.item()

#update map to save changes
vacc3_update = update_map(vacc_7d_map_id, vacc3_data)
vacc3_update

In [130]:
# TOTAL VACCINE & HPI Dashboard
#get map data
vacc4_data = get_map(vacc_hpi_id)

#adjust symbology for graduated points to reflect new max/min
#MAX VACC COUNT
# [7], [8], [9] --> size only, no color ramp
vacc4_data['operationalLayers'][7]['layerDefinition']['drawingInfo']['renderer']['authoringInfo']['visualVariables'][0]['maxSliderValue'] = max_count.item()
vacc4_data['operationalLayers'][7]['layerDefinition']['drawingInfo']['renderer']['visualVariables'][0]['maxDataValue'] = max_count.item()

vacc4_data['operationalLayers'][8]['layerDefinition']['drawingInfo']['renderer']['authoringInfo']['visualVariables'][0]['maxSliderValue'] = max_count.item()
vacc4_data['operationalLayers'][8]['layerDefinition']['drawingInfo']['renderer']['visualVariables'][0]['maxDataValue'] = max_count.item()

vacc4_data['operationalLayers'][9]['layerDefinition']['drawingInfo']['renderer']['authoringInfo']['visualVariables'][0]['maxSliderValue'] = max_count.item()
vacc4_data['operationalLayers'][9]['layerDefinition']['drawingInfo']['renderer']['visualVariables'][0]['maxDataValue'] = max_count.item()

#size info
vacc4_data['operationalLayers'][10]['layerDefinition']['drawingInfo']['renderer']['authoringInfo']['visualVariables'][0]['maxSliderValue'] = max_count.item()
vacc4_data['operationalLayers'][10]['layerDefinition']['drawingInfo']['renderer']['visualVariables'][1]['maxDataValue'] = max_count.item()
# color info
vacc4_data['operationalLayers'][10]['layerDefinition']['drawingInfo']['renderer']['authoringInfo']['visualVariables'][1]['maxSliderValue'] = max_rate.item()
vacc4_data['operationalLayers'][10]['layerDefinition']['drawingInfo']['renderer']['visualVariables'][0]['maxDataValue'] = max_rate.item()

#size info
vacc4_data['operationalLayers'][11]['layerDefinition']['drawingInfo']['renderer']['authoringInfo']['visualVariables'][0]['maxSliderValue'] = max_count.item()
vacc4_data['operationalLayers'][11]['layerDefinition']['drawingInfo']['renderer']['visualVariables'][1]['maxDataValue'] = max_count.item()
# color info
vacc4_data['operationalLayers'][11]['layerDefinition']['drawingInfo']['renderer']['authoringInfo']['visualVariables'][1]['maxSliderValue'] = max_rate.item()
vacc4_data['operationalLayers'][11]['layerDefinition']['drawingInfo']['renderer']['visualVariables'][0]['maxDataValue'] = max_rate.item()

#size info
vacc4_data['operationalLayers'][12]['layerDefinition']['drawingInfo']['renderer']['authoringInfo']['visualVariables'][0]['maxSliderValue'] = max_count.item()
vacc4_data['operationalLayers'][12]['layerDefinition']['drawingInfo']['renderer']['visualVariables'][0]['maxDataValue'] = max_count.item()
# color info
vacc4_data['operationalLayers'][12]['layerDefinition']['drawingInfo']['renderer']['authoringInfo']['visualVariables'][1]['maxSliderValue'] = max_rate.item()
vacc4_data['operationalLayers'][12]['layerDefinition']['drawingInfo']['renderer']['visualVariables'][1]['maxDataValue'] = max_rate.item()

#update map to save changes
vacc4_update = update_map(vacc_hpi_id, vacc4_data)
vacc4_update

<Item title:"COVID-19 Vaccines and Healthy Places Index (HPI)" type:Web Map owner:jembury8568_SDSUGeo>


True

In [ ]:
# 7-DAY VACCINE & HPI Dashboard
#get map data
vacc5_data = get_map(vacc_hpi_7d_id)

#adjust symbology for graduated points to reflect new max/min
#MAX VACC COUNT
# [7], [8], [9] --> size only, no color ramp
vacc5_data['operationalLayers'][7]['layerDefinition']['drawingInfo']['renderer']['authoringInfo']['visualVariables'][0]['maxSliderValue'] = max_7d.item()
vacc5_data['operationalLayers'][7]['layerDefinition']['drawingInfo']['renderer']['visualVariables'][0]['maxDataValue'] = max_7d.item()

vacc5_data['operationalLayers'][8]['layerDefinition']['drawingInfo']['renderer']['authoringInfo']['visualVariables'][0]['maxSliderValue'] = max_7d.item()
vacc5_data['operationalLayers'][8]['layerDefinition']['drawingInfo']['renderer']['visualVariables'][0]['maxDataValue'] = max_7d.item()

vacc5_data['operationalLayers'][9]['layerDefinition']['drawingInfo']['renderer']['authoringInfo']['visualVariables'][0]['maxSliderValue'] = max_7d.item()
vacc5_data['operationalLayers'][9]['layerDefinition']['drawingInfo']['renderer']['visualVariables'][0]['maxDataValue'] = max_7d.item()

#size info
vacc5_data['operationalLayers'][10]['layerDefinition']['drawingInfo']['renderer']['authoringInfo']['visualVariables'][0]['maxSliderValue'] = max_7d.item()
vacc5_data['operationalLayers'][10]['layerDefinition']['drawingInfo']['renderer']['visualVariables'][1]['maxDataValue'] = max_7d.item()
# color info
vacc5_data['operationalLayers'][10]['layerDefinition']['drawingInfo']['renderer']['authoringInfo']['visualVariables'][1]['maxSliderValue'] = max_7d_rate.item()
vacc5_data['operationalLayers'][10]['layerDefinition']['drawingInfo']['renderer']['visualVariables'][0]['maxDataValue'] = max_7d_rate.item()

#size info
vacc5_data['operationalLayers'][11]['layerDefinition']['drawingInfo']['renderer']['authoringInfo']['visualVariables'][0]['maxSliderValue'] = max_7d.item()
vacc5_data['operationalLayers'][11]['layerDefinition']['drawingInfo']['renderer']['visualVariables'][1]['maxDataValue'] = max_7d.item()
# color info
vacc5_data['operationalLayers'][11]['layerDefinition']['drawingInfo']['renderer']['authoringInfo']['visualVariables'][1]['maxSliderValue'] = max_7d_rate.item()
vacc5_data['operationalLayers'][11]['layerDefinition']['drawingInfo']['renderer']['visualVariables'][0]['maxDataValue'] = max_7d_rate.item()

#size info
vacc5_data['operationalLayers'][12]['layerDefinition']['drawingInfo']['renderer']['authoringInfo']['visualVariables'][0]['maxSliderValue'] = max_7d.item()
vacc5_data['operationalLayers'][12]['layerDefinition']['drawingInfo']['renderer']['visualVariables'][0]['maxDataValue'] = max_7d.item()
# color info
vacc5_data['operationalLayers'][12]['layerDefinition']['drawingInfo']['renderer']['authoringInfo']['visualVariables'][1]['maxSliderValue'] = max_7d_rate.item()
vacc5_data['operationalLayers'][12]['layerDefinition']['drawingInfo']['renderer']['visualVariables'][1]['maxDataValue'] = max_7d_rate.item()

#update map to save changes
vacc5_update = update_map(vacc_hpi_7d_id, vacc5_data)
vacc5_update

### VIEW UPDATED DASHBOARD

In [ ]:
webbrowser.open(vacc_dash, new=2)
webbrowser.open(vacc_7d_dash, new=2)
webbrowser.open(vacc_hpi_dash, new=2)
# webbrowser.open(vacc_hpi_7d_dash, new=2)